In [1]:
# Boiler plate stuff to start the module
import jpype
import jpype.imports
from jpype.types import *
import sys 
import numpy as np
import traceback
import random
import Client
import copy
import operator

C:\Users\AStA\anaconda3\envs\gym\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\AStA\anaconda3\envs\gym\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\AStA\anaconda3\envs\gym\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\AStA\anaconda3\envs\gym\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# import the Java modules
from java.io import IOException
from java.net import Socket
from java.net import UnknownHostException

from javax.net.ssl import SSLSocket
from javax.net.ssl import SSLSocketFactory
from javax.xml.bind import JAXBException
from javax.xml.bind import UnmarshalException

from de.fhac.mazenet.server.generated import AwaitMoveMessageData
from de.fhac.mazenet.server.generated import Errortype
from de.fhac.mazenet.server.generated import MazeCom
from de.fhac.mazenet.server.generated import MazeComMessagetype
from de.fhac.mazenet.server.generated import MoveMessageData
from de.fhac.mazenet.server.generated import BoardData

from de.fhac.mazenet.server.networking import MazeComMessageFactory
from de.fhac.mazenet.server.networking import XmlInputStream
from de.fhac.mazenet.server.networking import XmlOutputStream

from de.fhac.mazenet.server.game import Board
from de.fhac.mazenet.server.game import Position
from de.fhac.mazenet.server.game import Card

In [3]:
HOST = "127.0.0.1"
PORT = 9888

In [4]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

In [5]:
from mss import mss
import pydirectinput
import cv2
import pytesseract
import matplotlib.pyplot as plt
import time

In [6]:
import functools

from pettingzoo import AECEnv, ParallelEnv
from pettingzoo.utils import agent_selector, parallel_to_aec, wrappers

In [7]:
ls_range = np.arange(1,6,2)
potentialShiftMoves =  []
potentialShiftMovesPos =  []
for i in ls_range:
    potentialShiftMoves.append((0, i))
    potentialShiftMoves.append((6, i))
    potentialShiftMoves.append((i, 0))
    potentialShiftMoves.append((i, 6))
    potentialShiftMovesPos.append(Position(0, i))
    potentialShiftMovesPos.append(Position(6, i))
    potentialShiftMovesPos.append(Position(i, 0))
    potentialShiftMovesPos.append(Position(i, 6))

In [8]:
class MARL_Env(AECEnv):
    
    metadata = {"render_modes": ["human"], "name": "rps_v2"}

    def __init__(self, render_mode=None, num_players=4):
        self.num_players = num_players
        self.possible_agents = []
        self.agent_name_mapping = dict(
            zip(self.possible_agents, list(range(len(self.possible_agents))))
        )

        # Gym spaces are defined and documented here: https://gym.openai.com/docs/#spaces
        self._action_spaces = {agent: MultiDiscrete([12, 4, 7, 7], dtype=np.int8) for agent in self.possible_agents}
        self._observation_spaces = {
            agent: Tuple((Box(low=0, high=6, shape=(1,2), dtype=np.uint8),
                           Box(low=0, high=6, shape=(1,2), dtype=np.uint8)
                           ))  for agent in self.possible_agents
        }
        self.render_mode = render_mode
        self.player_pos = {agent: Box(low=0, high=6, shape=(1,2)) for agent in self.possible_agents}
        self.next_treasure_pos = {agent: Box(low=0, high=6, shape=(1,2)) for agent in self.possible_agents}
        self.is_done = False
        self.manager = Client.Client("manager", HOST, PORT, True)
        self.manager.login_manager()

    # this cache ensures that same space object is returned for the same agent
    # allows action space seeding to work as expected
    @functools.lru_cache(maxsize=None)
    def observation_space(self, agent):
        # Gym spaces are defined and documented here: https://gym.openai.com/docs/#spaces
        return Tuple((Box(low=0, high=6, shape=(1,2), dtype=np.uint8),
                           Box(low=0, high=6, shape=(1,2), dtype=np.uint8)
                           ))

    @functools.lru_cache(maxsize=None)
    def action_space(self, agent):
        return MultiDiscrete([12, 4, 7, 7], dtype=np.int8)

    def render(self):
        pass

    def observe(self, agent):
        """
        Observe should return the observation of the specified agent. This function
        should return a sane observation (though not necessarily the most up to date possible)
        at any time after reset() is called.
        """
        pass

    def close(self):
        """
        Close should release any graphical displays, subprocesses, network connections
        or any other environment data which should not be kept around after the
        user is no longer using the environment.
        """
        pass

    def reset(self, seed=None, return_info=False, options=None):
        """
        Reset needs to initialize the following attributes
        - agents
        - rewards
        - _cumulative_rewards
        - terminations
        - truncations
        - infos
        - agent_selection
        And must set up the environment so that render(), step(), and observe()
        can be called without issues.
        Here it sets up the state dictionary which is used by step() and the observations dictionary which is used by step() and observe()
        """
        self.manager.stop_game(self.num_players)
        self.manager.start_game(self.num_players)
        self.possible_agents = [Client.Client("player_" + str(r), HOST, PORT, True) for r in range(self.num_players)]
        
        self.agents = self.possible_agents[:]
        self.rewards = {agent: 0 for agent in self.agents}
        self._cumulative_rewards = {agent: 0 for agent in self.agents}
        self.terminations = {agent: False for agent in self.agents}
        self.truncations = {agent: False for agent in self.agents}
        self.infos = {agent: {} for agent in self.agents}
        self.state = {agent: None for agent in self.agents}
        self.observations = {agent: None for agent in self.agents}
        self.num_moves = 0
        """
        Our agent_selector utility allows easy cyclic stepping through the agents list.
        """
        self.is_done = False
        for agent in self.agents:
            while True:
                try:
                    login = MazeComMessageFactory.createLoginMessage(agent.name)
                    agent.out.write(login)
                    receivedMazeCom = agent.in_.readMazeCom()
                    if(receivedMazeCom.getMessagetype() == MazeComMessagetype.LOGINREPLY):
                        newid = receivedMazeCom.getLoginReplyMessage().getNewID()
                        print("newid:", newid)
                        agent.setId(newid)
                        break
                    else:
                        print(receivedMazeCom.getAcceptMessage().getErrortypeCode())
                except:
                    print("failed login :)")
        for i in self.agents:
            print("id:", i.id_)
        print("agents:", self.agents)
        self.agents = sorted(self.agents, key=operator.attrgetter('id_'))
        for i in self.agents:
            print("id:", i.id_)
        self._agent_selector = agent_selector(self.agents)
        self.agent_selection = self._agent_selector.next()
        print("end reset")
            

                
    def moeglicheOrientierungen(self, rot, card):
        card_ = Card(card)
        openings = card_.getOpenings()
        if(card.getShape() == Card.CardShape.I):
            openings.setBottom(not openings.isBottom())
            openings.setLeft(not openings.isLeft())
            openings.setRight(not openings.isRight())
            openings.setTop(not openings.isTop())
            return  Card(card_)
        else:
            if(rot == 0):
                return Card(card.getShape(), Card.Orientation.D0, card.getTreasure())
            if(rot == 1):
                return Card(card.getShape(), Card.Orientation.D90, card.getTreasure())
            if(rot == 2):
                return Card(card.getShape(), Card.Orientation.D180, card.getTreasure())
            if(rot == 3):
                return Card(card.getShape(), Card.Orientation.D270, card.getTreasure())
        return Card(card.getShape(), Card.Orientation.D0, card.getTreasure())
    
    def Move(self, awaitMove, action):
        reward = 0
        boardData = awaitMove.getBoard()
        treasure = awaitMove.getTreasureToFindNext()
        board = Board(boardData)
        playerPosition = board.findPlayer(self.agent_selection.getId())
        psm = potentialShiftMoves[action[0]]
        position = Position(psm[0], psm[1])
        
        potentialMove = MoveMessageData()
        psmp = copy.copy(potentialShiftMovesPos)
        if(position == board.getForbidden()):
            #print("forbidden shift position, random shift position used, reward decreased")
            reward -= 10
            try:
                psmp.remove(position)
            except Exception:
                print("element not in list, skipping")
            potentialMove.setShiftPosition(psmp[0])
        else:
            potentialMove.setShiftPosition(position)
        
        treasurePositionData = board.findTreasure(treasure)
        
        orientedShiftCard = self.moeglicheOrientierungen(action[1], board.getShiftCard())
        
        potentialMove.setShiftCard(orientedShiftCard)
        potentialMove.setNewPinPos(playerPosition)

        boardNext = board.fakeShift(potentialMove)
        treasurePositionData = boardNext.findTreasure(treasure)
        
        new_player_pos = boardNext.findPlayer(self.agent_selection.getId())
        reachablePositions = boardNext.getAllReachablePositions(new_player_pos)
        rp = np.array(reachablePositions)
        chosen_pos_obj = Position(action[2], action[3])
        chosen_pos_arr = np.array(action[2], action[3])
        if(treasurePositionData == None):
            #print("no treasure? skipping")
            potentialMove.setNewPinPos(new_player_pos)
            self.next_treasure_pos[self.agent_selection] = treasurePositionData
            self.player_pos[self.agent_selection] = new_player_pos
            return potentialMove, reward

        if(chosen_pos_obj not in rp):
            #print("cannot reach position, reward decreased")
            potentialMove.setNewPinPos(new_player_pos)
            self.next_treasure_pos[self.agent_selection] = treasurePositionData
            self.player_pos[self.agent_selection] = new_player_pos
            return potentialMove, reward - 10
        else:
            #print("treasure position data:", treasurePositionData)
            treasurePosition = Position(treasurePositionData)
            #print("treasure pos:", treasurePosition)

            if(chosen_pos_obj == treasurePosition):
                #print("straight to the treasure, reward increased!")
                potentialMove.setNewPinPos(treasurePosition)
                self.next_treasure_pos[self.agent_selection] = treasurePositionData
                self.player_pos[self.agent_selection] = treasurePosition
                return potentialMove, reward + 20
            else:
                #print("reachable position, but no treasure")
                potentialMove.setNewPinPos(chosen_pos_obj)
                self.next_treasure_pos[self.agent_selection] = treasurePositionData
                self.player_pos[self.agent_selection] = chosen_pos_obj
                return potentialMove, reward - 5

        #print("Object not reachable: random move :/")
        potentialMove.setNewPinPos(reachablePositions[0])
        self.next_treasure_pos[self.agent_selection] = treasurePositionData
        self.player_pos[self.agent_selection] = reachablePositions[0]
        return potentialMove, reward - 10
    
    def awaitMove(self, receivedMazeCom, action):
        awaitMove = receivedMazeCom.getAwaitMoveMessage()
        move, reward = self.Move(awaitMove, action)
        mazeComToSend = MazeCom()
        mazeComToSend.setId(self.agent_selection.getId())
        mazeComToSend.setMessagetype(MazeComMessagetype.MOVE)
        mazeComToSend.setMoveMessage(move)
        self.agent_selection.out.write(mazeComToSend)
        return reward                
                
    def step(self, action):
        """
        step(action) takes in an action for the current agent (specified by
        agent_selection) and needs to update
        - rewards
        - _cumulative_rewards (accumulating the rewards)
        - terminations
        - truncations
        - infos
        - agent_selection (to the next agent)
        And any internal state used by observe() or render()
        """
        print("begin step")
        if self.is_done:
            self.terminations[self.agent_selection] = True
        else:
            reward = 0
            try:
                receivedMazeCom = self.agent_selection.in_.readMazeCom()
                if(receivedMazeCom.getMessagetype() == MazeComMessagetype.LOGINREPLY):
                    newid = receivedMazeCom.getLoginReplyMessage().getNewID()
                    self.agent_selection.setId(newid)
                elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.ACCEPT):
                    print(receivedMazeCom.getAcceptMessage().getErrortypeCode())
                    reward += 2
                elif (receivedMazeCom.getMessagetype() == MazeComMessagetype.DISCONNECT):
                    print(receivedMazeCom.getDisconnectMessage().getErrortypeCode())
                    reward -= 10
                elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.AWAITMOVE):
                    print(action)
                    reward += self.awaitMove(receivedMazeCom, action)
                elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.MOVEINFO):
                    #print("in MoveInfo")
                    x = 0
                elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.WIN):
                    print("You have won")
                    self.is_done = True
                    reward += 100
                else:
                    print("Unknown message type: " + receivedMazeCom.getMessagetype())
            except Exception as e:
                print(traceback.format_exc())
                if str(type(e)) == "<java class 'java.net.SocketException'>":
                    print("socket error, disconnected")
                    sys.exit(1)
                elif str(type(e)) == "<java class 'java.io.EOFException'>":
                    print("end reached, disconnecting")
                    sys.exit(1)
            self.rewards[self.agent_selection] = reward
            self._cumulative_rewards[self.agent_selection] += reward
            self.observations[self.agent_selection] = (self.player_pos[self.agent_selection], self.next_treasure_pos[self.agent_selection])
            if self.is_done:
                self.terminations[self.agent_selection] = True
            else:
                self.terminations[self.agent_selection] = False
            self.agent_selection = self._agent_selector.next()


In [9]:
from pettingzoo.test import api_test
from pettingzoo.test import parallel_api_test

In [10]:
env = MARL_Env(num_players=4)
#api_test(env, num_cycles=100, verbose_progress=True)
env.reset()
#print(type(sorted(env.agent_iter(), key=operator.attrgetter('id_'))))
#print(env.agent_iter())
for agent in env.agent_iter():
    #print("agent:", agent.name)
    observation, reward, termination, truncation, info = env.last()
    action = env.action_space(agent).sample()
    # time.sleep(3)
    env.step(action)

run manager
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login

failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed l

failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed l

failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed l

failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed l

...

failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed l

failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed l

failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed l

failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed login :)
failed l


KeyboardInterrupt



In [ ]:
yx

In [ ]:
class MARL_Env_Parallel(ParallelEnv):
    
    metadata = {"render_modes": ["human"], "name": "rps_v2"}

    def __init__(self, render_mode=None):
        self.possible_agents = [Client.Client("player_" + str(r), HOST, PORT, False) for r in range(4)]
        self.agent_name_mapping = dict(
            zip(self.possible_agents, list(range(len(self.possible_agents))))
        )

        # Gym spaces are defined and documented here: https://gym.openai.com/docs/#spaces
        self._action_spaces = {agent: MultiDiscrete([12, 4, 7, 7], dtype=np.int8) for agent in self.possible_agents}
        self._observation_spaces = {
            agent: Tuple((Box(low=0, high=6, shape=(1,2), dtype=np.uint8),
                           Box(low=0, high=6, shape=(1,2), dtype=np.uint8)
                           ))  for agent in self.possible_agents
        }
        self.render_mode = render_mode
        self.player_pos = {agent: Box(low=0, high=6, shape=(1,2)) for agent in self.possible_agents}
        self.next_treasure_pos = {agent: Box(low=0, high=6, shape=(1,2)) for agent in self.possible_agents}
        self.is_done = False

    # this cache ensures that same space object is returned for the same agent
    # allows action space seeding to work as expected
    @functools.lru_cache(maxsize=None)
    def observation_space(self, agent):
        # Gym spaces are defined and documented here: https://gym.openai.com/docs/#spaces
        print("in the obs space")
        return Tuple((Box(low=0, high=6, shape=(1,2), dtype=np.uint8),
                           Box(low=0, high=6, shape=(1,2), dtype=np.uint8)
                           ))

    @functools.lru_cache(maxsize=None)
    def action_space(self, agent):
        return MultiDiscrete([12, 4, 7, 7], dtype=np.int8)

    def render(self):
        pass

    def observe(self, agent):
        """
        Observe should return the observation of the specified agent. This function
        should return a sane observation (though not necessarily the most up to date possible)
        at any time after reset() is called.
        """
        pass

    def close(self):
        """
        Close should release any graphical displays, subprocesses, network connections
        or any other environment data which should not be kept around after the
        user is no longer using the environment.
        """
        pass

    def reset(self, seed=None, return_info=False, options=None):
        """
        Reset needs to initialize the following attributes
        - agents
        - rewards
        - _cumulative_rewards
        - terminations
        - truncations
        - infos
        - agent_selection
        And must set up the environment so that render(), step(), and observe()
        can be called without issues.
        Here it sets up the state dictionary which is used by step() and the observations dictionary which is used by step() and observe()
        """
        self.agents = self.possible_agents[:]
        self.rewards = {agent: 0 for agent in self.agents}
        self._cumulative_rewards = {agent: 0 for agent in self.agents}
        self.terminations = {agent: False for agent in self.agents}
        self.truncations = {agent: False for agent in self.agents}
        self.infos = {agent: {} for agent in self.agents}
        self.state = {agent: None for agent in self.agents}
        self.observations = {agent: None for agent in self.agents}
        self.num_moves = 0
        """
        Our agent_selector utility allows easy cyclic stepping through the agents list.
        """
        self._agent_selector = agent_selector(self.agents)
        self.agent_selection = self._agent_selector.next()
        
        print("reeeset")
        for agent in self.agents:
            self.is_done = False
            try:
                login = MazeComMessageFactory.createLoginMessage(agent.name)
                agent.out.write(login)
            except:
                print("failed login :)")
            

                
    def moeglicheOrientierungen(self, rot, card):
        card_ = Card(card)
        openings = card_.getOpenings()
        if(card.getShape() == Card.CardShape.I):
            openings.setBottom(not openings.isBottom())
            openings.setLeft(not openings.isLeft())
            openings.setRight(not openings.isRight())
            openings.setTop(not openings.isTop())
            return  Card(card_)
        else:
            if(rot == 0):
                return Card(card.getShape(), Card.Orientation.D0, card.getTreasure())
            if(rot == 1):
                return Card(card.getShape(), Card.Orientation.D90, card.getTreasure())
            if(rot == 2):
                return Card(card.getShape(), Card.Orientation.D180, card.getTreasure())
            if(rot == 3):
                return Card(card.getShape(), Card.Orientation.D270, card.getTreasure())
        return Card(card.getShape(), Card.Orientation.D0, card.getTreasure())
    
    def Move(self, awaitMove, action):
        reward = 0
        boardData = awaitMove.getBoard()
        treasure = awaitMove.getTreasureToFindNext()
        board = Board(boardData)
        playerPosition = board.findPlayer(self.agent_selection.getId())
        psm = potentialShiftMoves[action[0]]
        position = Position(psm[0], psm[1])
        
        potentialMove = MoveMessageData()
        psmp = copy.copy(potentialShiftMovesPos)
        if(position == board.getForbidden()):
            print("forbidden shift position, random shift position used, reward decreased")
            reward -= 10
            try:
                psmp.remove(position)
            except Exception:
                print("element not in list, skipping")
            potentialMove.setShiftPosition(psmp[0])
        else:
            potentialMove.setShiftPosition(position)
        
        treasurePositionData = board.findTreasure(treasure)
        
        orientedShiftCard = self.moeglicheOrientierungen(action[1], board.getShiftCard())
        
        potentialMove.setShiftCard(orientedShiftCard)
        potentialMove.setNewPinPos(playerPosition)

        boardNext = board.fakeShift(potentialMove)
        treasurePositionData = boardNext.findTreasure(treasure)
        
        new_player_pos = boardNext.findPlayer(self.agent_selection.getId())
        reachablePositions = boardNext.getAllReachablePositions(new_player_pos)
        rp = np.array(reachablePositions)
        chosen_pos_obj = Position(action[2], action[3])
        chosen_pos_arr = np.array(action[2], action[3])
        if(treasurePositionData == None):
            print("no treasure? skipping")
            potentialMove.setNewPinPos(new_player_pos)
            self.next_treasure_pos = eval(treasurePositionData.toString())
            self.player_pos = eval(new_player_pos.toString())
            return potentialMove, reward

        if(chosen_pos_obj not in rp):
            print("cannot reach position, reward decreased")
            potentialMove.setNewPinPos(new_player_pos)
            self.next_treasure_pos = eval(treasurePositionData.toString())
            self.player_pos = eval(new_player_pos.toString())
            return potentialMove, reward - 10
        else:
            print("treasure position data:", treasurePositionData)
            treasurePosition = Position(treasurePositionData)
            print("treasure pos:", treasurePosition)

            if(chosen_pos_obj == treasurePosition):
                print("straight to the treasure, reward increased!")
                potentialMove.setNewPinPos(treasurePosition)
                self.next_treasure_pos = eval(treasurePositionData.toString())
                self.player_pos = eval(treasurePosition.toString())
                return potentialMove, reward + 20
            else:
                print("reachable position, but no treasure")
                potentialMove.setNewPinPos(chosen_pos_obj)
                self.next_treasure_pos = eval(treasurePositionData.toString())
                self.player_pos = eval(chosen_pos_obj.toString())
                return potentialMove, reward - 5

        print("Object not reachable: random move :/")
        potentialMove.setNewPinPos(reachablePositions[0])
        self.next_treasure_pos = eval(treasurePositionData.toString())
        self.player_pos = eval(reachablePositions[0].toString())
        return potentialMove, reward - 10
    
    def awaitMove(self, receivedMazeCom, action):
        awaitMove = receivedMazeCom.getAwaitMoveMessage()
        move, reward = self.Move(awaitMove, action)
        mazeComToSend = MazeCom()
        mazeComToSend.setId(self.agent_selection.getId())
        mazeComToSend.setMessagetype(MazeComMessagetype.MOVE)
        mazeComToSend.setMoveMessage(move)
        self.agent_selection.out.write(mazeComToSend)
        return reward                
                
    def step(self, action):
        """
        step(action) takes in an action for the current agent (specified by
        agent_selection) and needs to update
        - rewards
        - _cumulative_rewards (accumulating the rewards)
        - terminations
        - truncations
        - infos
        - agent_selection (to the next agent)
        And any internal state used by observe() or render()
        """
        print("begin step")
        print("agent", self.agent_selection.name)
        reward = 0
        print("ruun")
        try:
            print("test    1")
            receivedMazeCom = self.agent_selection.in_.readMazeCom()
            print("test    2")
            if(receivedMazeCom.getMessagetype() == MazeComMessagetype.LOGINREPLY):
                print("test    3")
                
                newid = receivedMazeCom.getLoginReplyMessage().getNewID()
                self.agent_selection.setId(newid)
            elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.ACCEPT):
                print("test    4")
                print(receivedMazeCom.getAcceptMessage().getErrortypeCode())
                reward += 2
            elif (receivedMazeCom.getMessagetype() == MazeComMessagetype.DISCONNECT):
                print("test    5")
                print(receivedMazeCom.getDisconnectMessage().getErrortypeCode())
                reward -= 10
            elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.AWAITMOVE):
                print("test    6")
                reward += self.awaitMove(receivedMazeCom, action)
            elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.MOVEINFO):
                print("in MoveInfo")
            elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.WIN):
                print("You have won")
                self.is_done = True
                reward += 100
            else:
                print("test   7")
                print("Unknown message type: " + receivedMazeCom.getMessagetype())
        except Exception as e:
            print("test    8")
            print(traceback.format_exc())
            if str(type(e)) == "<java class 'java.net.SocketException'>":
                print("socket error, disconnected")
                sys.exit(1)
            elif str(type(e)) == "<java class 'java.io.EOFException'>":
                print("end reached, disconnecting")
                sys.exit(1)
        print("test    9")
        self.rewards[self.agent_selection] = reward
        self.observations[self.agent_selection] = (self.player_pos[self.agent_selection], self.next_treasure_pos[self.agent_selection])
        if self.is_done:
            self.terminations[self.agent_selection] = True
        else:
            self.terminations[self.agent_selection] = False
        print("test    10")
        
        return self.observations, self.rewards, self.terminations, self.truncations, self.infos

In [ ]:
parallel_env = MARL_Env_Parallel()
observations = parallel_env.reset()
for step in range(100):
    actions = {agent: parallel_env.action_space(agent).sample() for agent in parallel_env.agents}  # this is where you would insert your policy
    observations, rewards, terminations, truncations, infos = parallel_env.step(actions)

In [ ]:
x

In [ ]:
menv = MazeEnv()

In [ ]:
menv.client = Client('joe', HOST, PORT, False)

In [ ]:
x

In [ ]:
try:
    c = Client('joe', HOST, PORT, False)
    c.login()
    c.play()
except Exception as e:
    print("type:", type(e))
    #print(traceback.format_exc())
    print("type:", type(e))

In [ ]:
xxc

In [ ]:
obs = menv.reset()
done = False  
total_reward   = 0
while not done:
    print("i = ", i)
    print(menv.action_space.sample())
    obs, reward,  done, info =  menv.step(menv.action_space.sample())
    #time.sleep(10)
    total_reward  += reward
    i += 1
print('Total Reward for episode {} is {}'.format(episode, total_reward))

In [ ]:
qa

In [ ]:
try:
    c = Client('joe', HOST, PORT, False)
    c.login()
    c.play()
except Exception:
    print(traceback.format_exc())

In [ ]:
qa